In [1]:
# import required libs
import glob
import os

import tensorflow as tf
import tensorflow_data_validation as tfdv
print('TF version: {}'.format(tf.version.VERSION))
print('TFDV version: {}'.format(tfdv.version.__version__))

2023-10-30 07:38:29.694141: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 07:38:29.731933: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-30 07:38:29.732666: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 07:38:30.648370: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


TF version: 2.13.1
TFDV version: 1.14.0


In [2]:
# Read artifact information from metadata store.
# import beam_dag_runner

from tfx.orchestration import metadata
from tfx.types import standard_artifacts
from ml_metadata.metadata_store.metadata_store import metadata_store_pb2

# NOTE: make sure to have forwarded to metadata grpc server in your development/prod server to connect to it.
# e.g. kubectl port-forward service/metadata-grpc-service 7000:8080
metadata_connection_config = metadata_store_pb2.MetadataStoreClientConfig()
metadata_connection_config.host = "localhost"
metadata_connection_config.port = 7000
with metadata.Metadata(metadata_connection_config) as store:
    stats_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleStatistics.TYPE_NAME)
    schema_artifacts = store.get_artifacts_by_type(standard_artifacts.Schema.TYPE_NAME)
    anomalies_artifacts = store.get_artifacts_by_type(standard_artifacts.ExampleAnomalies.TYPE_NAME)


In [3]:
filtered_stats = [x for x in stats_artifacts if "StatisticsGen" in x.uri]
filtered_schema = [x for x in schema_artifacts if "SchemaGen" in x.uri]


In [6]:
filtered_stats

[id: 2
 type_id: 19
 uri: "/local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/StatisticsGen/statistics/5"
 properties {
   key: "split_names"
   value {
     string_value: "[\"train\", \"eval\"]"
   }
 }
 custom_properties {
   key: "is_external"
   value {
     int_value: 0
   }
 }
 custom_properties {
   key: "stats_dashboard_link"
   value {
     string_value: ""
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.14.0"
   }
 }
 state: LIVE
 create_time_since_epoch: 1698334845186
 last_update_time_since_epoch: 1698334845186,
 id: 5
 type_id: 19
 uri: "/local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/StatisticsGen/statistics/10"
 properties {
   key: "split_names"
   value {
     string_value: "[\"train\", \"eval\"]"
   }
 }
 custom_properties {
   key: "is_external"
   value {
     int_value: 0
   }
 }
 custom_properties {
   key: "stats_dashboard_link"
   value {
     string_value: ""
   }
 }
 custom_properties {
   key: "tfx_version"
   

In [9]:
# configure output paths
# Exact paths to output artifacts can also be found on KFP Web UI if you are using kubeflow.
stats_path = filtered_stats[-1].uri
train_stats_file = os.path.join(stats_path, 'Split-train', 'FeatureStats.pb')
eval_stats_file = os.path.join(stats_path, 'Split-eval', 'FeatureStats.pb')
print("Train stats file:{}, Eval stats file:{}".format(
    train_stats_file, eval_stats_file))

schema_file = os.path.join(filtered_schema[-1].uri, 'schema.pbtxt')
print("Generated schame file:{}".format(schema_file))
anomalies_file = os.path.join(anomalies_artifacts[-1].uri, 'anomalies.pbtxt')
print("Generated anomalies file:{}".format(anomalies_file))

Train stats file:/local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/StatisticsGen/statistics/20/Split-train/FeatureStats.pb, Eval stats file:/local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/StatisticsGen/statistics/20/Split-eval/FeatureStats.pb
Generated schame file:/local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/SchemaGen/schema/21/schema.pbtxt
Generated anomalies file:/local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/Transform/post_transform_anomalies/22/anomalies.pbtxt


In [11]:
tfdv.load_stats_binary("/local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/StatisticsGen/statistics/20/Split-eval/FeatureStats.pb")

NotFoundError: /local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/StatisticsGen/statistics/20/Split-eval/FeatureStats.pb; No such file or directory

In [8]:
# load generated statistics from StatisticsGen
train_stats = tfdv.load_stats_binary(train_stats_file)
eval_stats = tfdv.load_stats_binary(eval_stats_file)
tfdv.visualize_statistics(lhs_statistics=train_stats, rhs_statistics=eval_stats,
                          lhs_name='TRAIN_DATASET', rhs_name="EVAL_DATASET")

NotFoundError: /local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/StatisticsGen/statistics/20/train/FeatureStats.pb; No such file or directory

In [12]:
# load generated schema from SchemaGen
schema = tfdv.load_schema_text(schema_file)
tfdv.display_schema(schema=schema)

NotFoundError: /local/taxi/artifacts/tfx_pipeline_output/kubeflow-taxi/SchemaGen/schema/21/schema.pbtxt; No such file or directory

In [ ]:
# load data vaildation result from ExampleValidator
anomalies = tfdv.load_anomalies_text(anomalies_file)
tfdv.display_anomalies(anomalies)